In [2]:
import numpy as np
import pandas as pd

import torch
import torch.autograd
from torch.autograd import Variable

df_iris = pd.read_csv(u'data/iris.txt',sep=' ')

eta = 0.0001

trials = 1000

In [11]:
def f(x, w):
    return x.dot(w)

def E(y, f_est):
    return np.sum((y-f_est)**2)

x1 = np.array(df_iris[['sl', 'sw', 'pl', 'pw']])

y1 = np.array(df_iris['c'])

w1 = np.array([1.,1.,1.,1.])

for epoch in range(trials):
    f_est = f(x1, w1)
    e = y1 - f_est
    dE = -x1.T.dot(e)
    
    w1 = w1 - eta * dE
    
result1 = np.round(f(x1, w1))

confusion_matrix1 = np.zeros((3,3))

for i in range(len(result1)):
    confusion_matrix1[int(y1[i] - 1), int(result1[i] - 1)] += 1
    
accuracy1 = np.sum(np.diag(confusion_matrix1))/np.sum(confusion_matrix1)

print("Linear Regression W\n", w1)
print("Linear Regression Confusion Matrix\n", confusion_matrix1)
print("Linear Regression Accuracy\n", accuracy1*100, "%")

Linear Regression W
 [-0.01245586  0.16331546  0.22548113  0.59811822]
Linear Regression Confusion Matrix
 [[ 50.   0.   0.]
 [  0.  48.   2.]
 [  0.   4.  46.]]
Linear Regression Accuracy
 96.0 %


In [12]:
def logsumexp(b):
    return torch.log(1 + torch.exp(b))

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

y2 = torch.from_numpy(np.array(df_iris['c'])).double()
y2[y2>1]=0

x2 = torch.from_numpy(np.array(df_iris[['sl','sw','pl','pw']])).double()

ws = Variable(torch.randn(4).double(), requires_grad=True)

for epoch in range(trials):
    for i in range(len(y2)):
        f = torch.matmul(x2[i,:],ws)
        
        E = -y2[i]*f + logsumexp(f)

        E.backward()

    ws.data.add_(-eta * ws.grad.data)
    
    ws.grad.zero_()
    
w2 = ws.data

result2 = torch.round(sigmoid(torch.matmul(x2,w2)))

confusion_matrix2 = torch.zeros((2,2))

for i in range(len(result2)):
    confusion_matrix2[int(y2[i] - 1), int(result2[i] - 1)] += 1
    
accuracy2 = torch.sum(torch.diag(confusion_matrix2))/torch.sum(confusion_matrix2)

print("Logical Regression W:\n", w2)
print("Logical Regression Confusion Matrix:\n", confusion_matrix2)
print("Logical Regression Accuracy:\n", accuracy2*100, "%")

Logical Regression W:
 tensor([-1.1825,  3.1540, -1.0975, -0.4948], dtype=torch.float64)
Logical Regression Confusion Matrix:
 tensor([[ 50.,   0.],
        [  0., 100.]])
Logical Regression Accuracy:
 tensor(100.) %
